In [1]:
import os
import sys
curruser = os.environ.get('USER')

_labdata = os.environ.get("LABDATA_PYSPARK")
sys.path.insert(0, _labdata)
os.chdir(_labdata)

if curruser in os.listdir("/opt/workspace/"):
    sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
    # sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata_v1.2/lib/'.format(user=curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser))
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    # sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime as dt

from transliterate import translit

from lib.spark_connector import SparkConnector
from lib.sparkdb_loader import *
from lib.connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.utils import AnalysisException

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from collections import ChainMap

from lib.config import *
from lib.tools import *

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))

def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

    return _typed_udf_wrapper

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


## Generate New Kerberos Ticket from PreSaved User Pass

In [3]:
!PASS=$(cat /home/$(whoami)/pass/userpswrd | sed 's/\r//g'); kdestroy && echo $PASS | kinit

Password for ektov1-av_ca-sbrf-ru@DF.SBRF.RU: 


In [4]:
sp = spark(schema=CONN_SCHEMA,
               dynamic_alloc=False,
               numofinstances=5,
               numofcores=8,
               executor_memory='15g',
               driver_memory='15g',
               kerberos_auth=False,
               process_label="_CHECK_HDFS_"
               )

hive = sp.sql
print(sp.sc.version)

====================================================== 2022-01-29 ======================================================
# __init__ : begin
2.4.0.cloudera2


In [8]:
CONN_SCHEMA = 'sbx_t_team_cvm' #'sbx_team_digitcamp' #'sbx_t_team_cvm'

In [6]:
hive.setConf("hive.exec.dynamic.partition","true")
hive.setConf("hive.exec.dynamic.partition.mode","nonstrict")
hive.setConf("hive.enforce.bucketing","false")
hive.setConf("hive.enforce.sorting","false")
# hive.setConf("hive.exec.stagingdir", "/tmp/{}/".format(curruser))
# hive.setConf("hive.exec.scratchdir", "/tmp/{}/".format(curruser))
hive.setConf("hive.load.dynamic.partitions.thread", 1)

## Following command generates output log file after running dfs -du command follwing with human-like sorting by size columns

In [ ]:
!hdfs dfs -du -s -h hdfs://clsklsbx/user/team/team_cvm/hive/* | awk '{print $1$2, $3$4, $5}' | sort -h -r -k2 >> ~/hdfs_consump_log

## Generate statistics using `describe extended` command repetitively

In [9]:
def table_size(hdfs_path):
    try:
        p = subprocess.Popen(['hdfs', 'dfs', '-du', '-s', hdfs_path], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
        res= re.sub("\s+"," ",p.communicate()[0].decode('utf-8').rsplit(" ",1)[0]).strip().split(" ")
        size, size_fr = ["{:10.4f} Gb".format(ele) for ele in np.array(list(map(float, res)))/1.0e9]
    except ValueError:
        size, size_fr = [None]*2
    return size, size_fr

In [10]:
hdfs_dct={'OWNER':[],
          'HDFS_PATH':[],
          'SIZE':[],
          'SIZE_FR':[]}

dbs_row = hive.sql("show tables in {}".format(CONN_SCHEMA)).collect()
dbs_lst = [item['tableName'] for item in dbs_row]

for dbs_name in dbs_lst:
    try:
        descr = hive.sql("describe formatted {}.{}".format(CONN_SCHEMA, dbs_name)).collect()
        descr_info = [item.asDict() for item in descr if item.asDict()['col_name'] =='Location']
        owner_info = [item.asDict() for item in descr if item.asDict()['col_name'] =='Owner']
        hdfs_path  = dict(ChainMap(*descr_info)).get('data_type',None)
        owner      = dict(ChainMap(*owner_info)).get('data_type',None).split('@')[0]
        if hdfs_path is not None:
            hdfs_path_to_tbl = os.path.join(hdfs_path.rsplit('/',1)[0], dbs_name)
            print(hdfs_path_to_tbl, owner)

            size, size_fr = table_size(hdfs_path_to_tbl)
            print("Table: {} # HDFS Consumed Space --> {}".format(dbs_name, size))
            hdfs_dct['OWNER'].append(owner)
            hdfs_dct['HDFS_PATH'].append(hdfs_path_to_tbl)
            hdfs_dct['SIZE'].extend([size.strip() if size is not None else None])
            hdfs_dct['SIZE_FR'].extend([size_fr.strip() if size_fr is not None else None])
            
    except AnalysisException as ex:
        print(str(ex))

hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_09112021 okhilkov-as_ca-sbrf-ru
Table: acq_data_09112021 # HDFS Consumed Space -->     0.0602 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_09112021_prep geraskin1-iv_ca-sbrf-ru
Table: acq_data_09112021_prep # HDFS Consumed Space -->     0.4975 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_21102021 okhilkov-as_ca-sbrf-ru
Table: acq_data_21102021 # HDFS Consumed Space -->     0.0597 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_25082021 okhilkov-as_ca-sbrf-ru
Table: acq_data_25082021 # HDFS Consumed Space -->     0.0582 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_25082021_prep okhilkov-as_ca-sbrf-ru
Table: acq_data_25082021_prep # HDFS Consumed Space -->     0.4574 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_data_30062021 okhilkov-as_ca-sbrf-ru
Table: acq_data_30062021 # HDFS Consumed Space -->     0.0568 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/acq_v0_1_churn_target okhilkov-as_ca-sbrf-ru
Table: acq_v0_1_ch

Table: all_features_for_lal_auto_with_tsfresh_0 # HDFS Consumed Space -->    27.1574 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/all_features_for_lal_auto_with_tsfresh_6 maltsev2-ay_ca-sbrf-ru
Table: all_features_for_lal_auto_with_tsfresh_6 # HDFS Consumed Space -->    11.7422 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/all_features_for_lal_auto_with_tsfresh_all maltsev2-ay_ca-sbrf-ru
Table: all_features_for_lal_auto_with_tsfresh_all # HDFS Consumed Space -->    36.0052 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/all_features_for_lal_old shaymanov-an_ca-sbrf-ru
Table: all_features_for_lal_old # HDFS Consumed Space -->     6.3370 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/analog_u_crm_data okhilkov-as_ca-sbrf-ru
Table: analog_u_crm_data # HDFS Consumed Space -->     3.4485 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/bsa_als_deals_test bulaev-sa_ca-sbrf-ru
Table: bsa_als_deals_test # HDFS Consumed Space -->     3.3411 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/bsa_als_deals_train bulaev-sa_c

Table: ch_b30_model_test_2020_10_31 # HDFS Consumed Space -->     0.0668 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_3or4or5_mmb geraskin1-iv_ca-sbrf-ru
Table: ch_b30_target_3or4or5_mmb # HDFS Consumed Space -->     0.3936 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_3or4or5_mmb_sample geraskin1-iv_ca-sbrf-ru
Table: ch_b30_target_3or4or5_mmb_sample # HDFS Consumed Space -->     0.0554 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_lag_0_lw_2 okhilkov-as_ca-sbrf-ru
Table: ch_b30_target_lag_0_lw_2 # HDFS Consumed Space -->     0.5033 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_lag_0_lw_2_mmb okhilkov-as_ca-sbrf-ru
Table: ch_b30_target_lag_0_lw_2_mmb # HDFS Consumed Space -->     0.4220 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_lag_0_lw_2_mmb_sample okhilkov-as_ca-sbrf-ru
Table: ch_b30_target_lag_0_lw_2_mmb_sample # HDFS Consumed Space -->     0.0296 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_target_lag_0_lw_3 okhilkov-

Table: ch_b30_v1_3_na_bug_target_lag_0_lw_5 # HDFS Consumed Space -->    10.2376 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_v1_3_na_bug_target_lag_0_lw_5_mmb okhilkov-as_ca-sbrf-ru
Table: ch_b30_v1_3_na_bug_target_lag_0_lw_5_mmb # HDFS Consumed Space -->     8.9826 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_v1_3_na_bug_target_lag_0_lw_5_mmb_sample okhilkov-as_ca-sbrf-ru
Table: ch_b30_v1_3_na_bug_target_lag_0_lw_5_mmb_sample # HDFS Consumed Space -->     0.5911 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_v1_3_pilot_data_mmb_lag_0_mon_2021_04_30 okhilkov-as_ca-sbrf-ru
Table: ch_b30_v1_3_pilot_data_mmb_lag_0_mon_2021_04_30 # HDFS Consumed Space -->     1.3173 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_v1_3_preds_refit_pilot_2021_04_30 okhilkov-as_ca-sbrf-ru
Table: ch_b30_v1_3_preds_refit_pilot_2021_04_30 # HDFS Consumed Space -->     0.0507 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/ch_b30_v1_3_preds_refit_pilot_sigm_2021_04_30 okhilkov-as_ca-sbrf-ru
Table: ch_b

Table: churn_acq_tsf_orig_val_na_mean_short_id_tst_2 # HDFS Consumed Space -->     0.4227 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/churn_acq_tsf_orig_val_na_mean_v1_3 okhilkov-as_ca-sbrf-ru
Table: churn_acq_tsf_orig_val_na_mean_v1_3 # HDFS Consumed Space -->     7.4944 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/churn_acq_tsf_orig_val_na_mean_v1_3_salary okhilkov-as_ca-sbrf-ru
Table: churn_acq_tsf_orig_val_na_mean_v1_3_salary # HDFS Consumed Space -->     9.7081 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/churn_acq_tsf_roll_val_na_zero okhilkov-as_ca-sbrf-ru
Table: churn_acq_tsf_roll_val_na_zero # HDFS Consumed Space -->     0.0696 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/churn_acq_tsf_roll_val_na_zero_short_id okhilkov-as_ca-sbrf-ru
Table: churn_acq_tsf_roll_val_na_zero_short_id # HDFS Consumed Space -->     0.2090 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/churn_acquiring_activity_features_cnt_acq_v_2 okhilkov-as_ca-sbrf-ru
Table: churn_acquiring_activity_features_cnt_acq_v_2 # HDF

...

Table: v1_4_katya_2019_model_100_imp_features_2019_01_2020_10_full_set_predictions_2021_01_31 # HDFS Consumed Space -->     0.0340 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_4_katya_2019_model_100_imp_features_2019_01_2020_10_full_set_predictions_2021_02_28 kharlova-ey_ca-sbrf-ru
Table: v1_4_katya_2019_model_100_imp_features_2019_01_2020_10_full_set_predictions_2021_02_28 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_4_pilot_data_mmb_mon_2021_04_30 geraskin1-iv_ca-sbrf-ru
Table: v1_4_pilot_data_mmb_mon_2021_04_30 # HDFS Consumed Space -->     1.6745 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_4_testtarget_churn_target kharlova-ey_ca-sbrf-ru
Table: v1_4_testtarget_churn_target # HDFS Consumed Space -->     0.5672 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_4_testtarget_churn_target_mmb kharlova-ey_ca-sbrf-ru
Table: v1_4_testtarget_churn_target_mmb # HDFS Consumed Space -->     0.5836 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_churn_target

Table: v1_5_full_set_predictions_2020_11_30_top10 # HDFS Consumed Space -->     0.0342 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_full_set_predictions_2020_12_31 okhilkov-as_ca-sbrf-ru
Table: v1_5_full_set_predictions_2020_12_31 # HDFS Consumed Space -->     0.0342 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_full_set_predictions_2020_12_31_short_train geraskin1-iv_ca-sbrf-ru
Table: v1_5_full_set_predictions_2020_12_31_short_train # HDFS Consumed Space -->     0.0908 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_full_set_predictions_2020_12_31_top10 okhilkov-as_ca-sbrf-ru
Table: v1_5_full_set_predictions_2020_12_31_top10 # HDFS Consumed Space -->     0.0342 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_full_set_predictions_2021_01_31_top10 okhilkov-as_ca-sbrf-ru
Table: v1_5_full_set_predictions_2021_01_31_top10 # HDFS Consumed Space -->     0.0340 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_full_set_predictions_2021_02_28_top10 okhilkov-as_ca-sbrf-ru
Table: v1_5_full_se

Table: v1_5_our_fall_dohod_full_set_predictions__02_19_06_20_top100_2020_12_31 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_our_fall_dohod_full_set_predictions_2020_09_30 geraskin1-iv_ca-sbrf-ru
Table: v1_5_our_fall_dohod_full_set_predictions_2020_09_30 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_our_fall_dohod_full_set_predictions_2020_10_31 geraskin1-iv_ca-sbrf-ru
Table: v1_5_our_fall_dohod_full_set_predictions_2020_10_31 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_our_fall_dohod_full_set_predictions_2020_11_30 geraskin1-iv_ca-sbrf-ru
Table: v1_5_our_fall_dohod_full_set_predictions_2020_11_30 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v1_5_our_fall_dohod_full_set_predictions_2020_12_31 geraskin1-iv_ca-sbrf-ru
Table: v1_5_our_fall_dohod_full_set_predictions_2020_12_31 # HDFS Consumed Space -->     0.0341 Gb
hdfs://clsklsbx/user/

Table: v2_1_pilot_19_11_pilot_data_mmb_mon_part2_2021_07_31 # HDFS Consumed Space -->     0.3560 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_19_11_pilot_data_mmb_mon_part2_2021_10_31 geraskin1-iv_ca-sbrf-ru
Table: v2_1_pilot_19_11_pilot_data_mmb_mon_part2_2021_10_31 # HDFS Consumed Space -->     0.3627 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_19_11_pilot_data_mmb_mon_part3_2021_07_31 geraskin1-iv_ca-sbrf-ru
Table: v2_1_pilot_19_11_pilot_data_mmb_mon_part3_2021_07_31 # HDFS Consumed Space -->     0.1095 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_19_11_pilot_data_mmb_mon_part3_2021_10_31 geraskin1-iv_ca-sbrf-ru
Table: v2_1_pilot_19_11_pilot_data_mmb_mon_part3_2021_10_31 # HDFS Consumed Space -->     0.1110 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_19_11_pilot_data_mmb_mon_union_2021_07_31 geraskin1-iv_ca-sbrf-ru
Table: v2_1_pilot_19_11_pilot_data_mmb_mon_union_2021_07_31 # HDFS Consumed Space -->     1.9867 Gb
hdfs://clsklsbx/user/team/team_c

Table: v2_1_pilot_full_set_predictions_all_feat_2021_01_31 # HDFS Consumed Space -->     0.0293 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_full_set_predictions_all_feat_2021_02_28 okhilkov-as_ca-sbrf-ru
Table: v2_1_pilot_full_set_predictions_all_feat_2021_02_28 # HDFS Consumed Space -->     0.0294 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_full_set_predictions_all_feat_2021_03_31 okhilkov-as_ca-sbrf-ru
Table: v2_1_pilot_full_set_predictions_all_feat_2021_03_31 # HDFS Consumed Space -->     0.0293 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_full_set_predictions_fs_ho_2020_11_30 okhilkov-as_ca-sbrf-ru
Table: v2_1_pilot_full_set_predictions_fs_ho_2020_11_30 # HDFS Consumed Space -->     0.0343 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_full_set_predictions_fs_ho_2020_12_31 okhilkov-as_ca-sbrf-ru
Table: v2_1_pilot_full_set_predictions_fs_ho_2020_12_31 # HDFS Consumed Space -->     0.0343 Gb
hdfs://clsklsbx/user/team/team_cvm/hive/v2_1_pilot_full_s

In [11]:
df = pd.DataFrame(hdfs_dct)
new = df.assign(f = lambda x: x['SIZE_FR'].apply(lambda y: float(y.split('Gb')[0]) if y is not None else y))\
        .sort_values(by='f', ascending=False).drop('f',axis=1).reset_index(drop=True)
    
new.head(20)

,HDFS_PATH,OWNER,SIZE,SIZE_FR
0,hdfs://clsklsbx/user/team/team_cvm/hive/txn_20...,tumanov1-ga_ca-sbrf-ru,1462.8132 Gb,2925.6264 Gb
1,hdfs://clsklsbx/user/team/team_cvm/hive/mon_ai...,zhernokleev-ga_ca-sbrf-ru,1035.6479 Gb,2071.2959 Gb
2,hdfs://clsklsbx/user/team/team_cvm/hive/client...,kovaleva4-oa_ca-sbrf-ru,507.7636 Gb,1024.8576 Gb
3,hdfs://clsklsbx/user/team/team_cvm/hive/ok_smz...,kovaleva4-oa_ca-sbrf-ru,276.9249 Gb,553.8499 Gb
4,hdfs://clsklsbx/user/team/team_cvm/hive/mmb_of...,ektov1-av_ca-sbrf-ru,165.4934 Gb,330.9868 Gb
5,hdfs://clsklsbx/user/team/team_cvm/hive/ma_mmb...,ektov1-av_ca-sbrf-ru,164.1438 Gb,328.2877 Gb
6,hdfs://clsklsbx/user/team/team_cvm/hive/churn_...,geraskin1-iv_ca-sbrf-ru,86.4890 Gb,259.4670 Gb
7,hdfs://clsklsbx/user/team/team_cvm/hive/churn_...,geraskin1-iv_ca-sbrf-ru,80.7679 Gb,242.3038 Gb
8,hdfs://clsklsbx/user/team/team_cvm/hive/churn_...,geraskin1-iv_ca-sbrf-ru,80.7629 Gb,242.2886 Gb
9,hdfs://clsklsbx/user/team/team_cvm/hive/mmb_of...,ektov1-av_ca-sbrf-ru,120.9090 Gb,241.8180 Gb


In [12]:
new.iloc[0:7].values

array([['hdfs://clsklsbx/user/team/team_cvm/hive/txn_2020_12',
        'tumanov1-ga_ca-sbrf-ru', '1462.8132 Gb', '2925.6264 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/mon_ai_fl_embedding_v3',
        'zhernokleev-ga_ca-sbrf-ru', '1035.6479 Gb', '2071.2959 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/clients_products_extract',
        'kovaleva4-oa_ca-sbrf-ru', '507.7636 Gb', '1024.8576 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/ok_smz_dataset',
        'kovaleva4-oa_ca-sbrf-ru', '276.9249 Gb', '553.8499 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/mmb_offer_proc_coe',
        'ektov1-av_ca-sbrf-ru', '165.4934 Gb', '330.9868 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/ma_mmb_offer_nontop',
        'ektov1-av_ca-sbrf-ru', '164.1438 Gb', '328.2877 Gb'],
       ['hdfs://clsklsbx/user/team/team_cvm/hive/churn_seasonal_features_v2_1_pilot_19_11',
        'geraskin1-iv_ca-sbrf-ru', '86.4890 Gb', '259.4670 Gb']],
      dtype=object)

In [15]:
new.to_excel("~/hdfs_team_cvm_disk_usage.xlsx")

## Make list of tables for purging

In [ ]:
## tbls =[]

## Drop tables recursively

In [ ]:
for table_name in tbls:
    hive.sql("drop table if exists {schema}.{tbl} purge".format(schema=CONN_SCHEMA, tbl=table_name))
    subprocess.call(['hdfs', 'dfs', '-rm', '-R', '-r', '-skipTrash', 
                     "hdfs://clsklsbx/user/team/team_cvm/hive/{}".format(table_name)])

## Calculate HDFS disk consumption

In [13]:
!hdfs dfs -du hdfs://clsklsbx/user/team/team_cvm/hive/ | awk '{s+=$2}END{print s/1000000000}'

17083.5
